## Imports

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
import itertools
import random
import warnings
import numpy as np
import matplotlib.pyplot as plt
import cv2
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint, EarlyStopping
import sklearn as sk
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
warnings.simplefilter(action='ignore', category=FutureWarning)

## Train and Test image directory paths

In [ ]:
train_path = r'D:\gesture\train'
test_path = r'D:\gesture\test'

In [ ]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=train_path, target_size=(64,64), class_mode='categorical', batch_size=10, shuffle=True)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input).flow_from_directory(directory=test_path, target_size=(64,64), class_mode='categorical', batch_size=10, shuffle=True)

## Building the model

In [ ]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(64,64,3)))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding = 'same'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding = 'valid'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))

model.add(Flatten())

model.add(Dense(64,activation ="relu"))
model.add(Dense(128,activation ="relu"))
model.add(Dense(128,activation ="relu"))
model.add(Dense(3,activation ="softmax"))

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=0.0001)
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')

model.compile(optimizer=SGD(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=0.0005)
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')

history2 = model.fit(train_batches, epochs=100, callbacks=[reduce_lr, early_stop],  validation_data = test_batches)

In [ ]:
history2.history

## Saving our model

In [ ]:
model.save('sign_model.h5')

## Metrics

In [ ]:
imgs, labels = next(test_batches) 
scores = model.evaluate(imgs, labels, verbose=0)
print('{0} of {1}; {2} of {3}%'
.format(model.metrics_names[0], scores[0], model.metrics_names[1], scores[1]*100))
scores

In [ ]:
model.reca

In [ ]:
imgs, labels = next(test_batches)
model = keras.models.load_model(r"sign_model.h5")
scores = model.evaluate(imgs, labels, verbose=0)
print('{0} of {1}; {2} of {3}%'
.format(model.metrics_names[0], scores[0], model.metrics_names[1], scores[1]*100))

In [ ]:
model.summary()

## Predictions

In [ ]:
imgs, labels = next(test_batches) 

word_dict = {0:'One', 1:'Two', 2:'Three'}

predictions = model.predict(imgs, verbose=0)
predicted_labels = []
actual_labels = []

print("Predicted labels")
for ind, i in enumerate(predictions):
    print(word_dict[np.argmax(i)])
    predicted_labels.append(word_dict[np.argmax(i)])
    
print('Actual labels')
for i in labels:
    print(word_dict[np.argmax(i)])
    actual_labels.append(word_dict[np.argmax(i)])


In [ ]:
print ("Precision", sk.metrics.precision_score(actual_labels, predicted_labels, average='macro'))
print ("Recall", sk.metrics.recall_score(actual_labels, predicted_labels, average='macro'))
print ("f1_score", sk.metrics.f1_score(actual_labels, predicted_labels, average='macro'))
print ("confusion_matrix: ")
print (sk.metrics.confusion_matrix(actual_labels, predicted_labels))

In [ ]:
labels

In [ ]:
predictions